In [1]:
import torch
import torch.nn as nn
from timeit import default_timer as timer
from tqdm import tqdm

In [2]:
images = torch.randn(size=(32,3,28,28))
test_image = images[0]
channel, length, width = test_image.shape

In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# Setup training data
train_data = datasets.FashionMNIST(
    root="data", # where to download data to?
    train=True, # get training data
    download=True, # download data if it doesn't exist on disk
    transform=ToTensor(), # images come as PIL format, we want to turn into Torch tensors
    target_transform=None # you can transform labels as well
)

# Setup testing data
test_data = datasets.FashionMNIST(
    root="data",
    train=False, # get test data
    download=True,
    transform=ToTensor()
)



# Setup the batch size hyperparameter
BATCH_SIZE = 32

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_data, # dataset to turn into iterable
    batch_size=BATCH_SIZE, # how many samples per batch?
    shuffle=True # shuffle data every epoch?
)

test_dataloader = DataLoader(test_data,
    batch_size=BATCH_SIZE,
    shuffle=False # don't necessarily have to shuffle the testing data
)

100%|██████████| 26421880/26421880 [00:01<00:00, 14917183.48it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 264394.69it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4990336.92it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 16961725.84it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
class MNISTModel(nn.Module):
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
    super().__init__()
    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  padding=1,
                  stride=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  padding=1,
                  stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv_block2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  padding=1,
                  stride=1),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=hidden_units,
            out_channels=hidden_units,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*7*7,
                  out_features=output_shape)
    )

  def forward(self,x):
    x = self.conv_block1(x)
    #print(x.shape)
    x = self.conv_block2(x)
    #print(x.shape)
    x = self.classifier(x)
    #print(x.shape)
    return x

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
model = MNISTModel(input_shape=1,
                   hidden_units=10,
                   output_shape=10)

In [7]:
# Loss functions
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.01)

In [8]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [9]:
def train_step(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    accuracy_fn,
    device:torch.device = device
):
  train_loss,train_acc =0,0
  for batch,(X,Y) in enumerate(data_loader):
    X,Y = X.to(device),Y.to(device)
    Y_pred = model(X)

    loss = loss_fn(Y_pred,Y)
    train_loss+=loss
    train_acc+=accuracy_fn(y_true=Y,
                           y_pred=Y_pred.argmax(dim=1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Calculate loss and accuracy per epoch and print out what's happening
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")



In [10]:
def test_step(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    accuracy_fn,
    device:torch.device = device
):
  test_loss,test_acc =0,0
  with torch.inference_mode():
    for batch,(X,Y) in enumerate(data_loader):
      X,Y = X.to(device),Y.to(device)
      Y_pred = model(X)

      loss = loss_fn(Y_pred,Y)
      test_loss+=loss
      test_acc+=accuracy_fn(y_true=Y,
                            y_pred=Y_pred.argmax(dim=1))


    # Calculate loss and accuracy per epoch and print out what's happening
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
  print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%")

In [11]:
from timeit import default_timer as timer
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format).
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [12]:
# training step
torch.manual_seed(42)


train_time_start_model = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
  print(f"--------------- Epoch : {epoch} ---------------")
  train_step(model=model,
             data_loader=train_dataloader,
             loss_fn=loss_fn,
             accuracy_fn=accuracy_fn,
             device=device,
             optimizer=optimizer)
  test_step(model=model,
             data_loader=test_dataloader,
             loss_fn=loss_fn,
             accuracy_fn=accuracy_fn,
             device=device)

train_time_stop_model = timer()
total_train_time_model_1 = print_train_time(start=train_time_start_model,
                                            end=train_time_stop_model,
                                            device=device)

  0%|          | 0/3 [00:00<?, ?it/s]

--------------- Epoch : 0 ---------------
Train loss: 1.34375 | Train accuracy: 52.16%


 33%|███▎      | 1/3 [01:06<02:12, 66.08s/it]

Test loss: 0.65307 | Test accuracy: 76.30%
--------------- Epoch : 1 ---------------
Train loss: 0.56991 | Train accuracy: 79.45%


 67%|██████▋   | 2/3 [02:04<01:01, 61.57s/it]

Test loss: 0.50522 | Test accuracy: 81.67%
--------------- Epoch : 2 ---------------
Train loss: 0.45889 | Train accuracy: 83.56%


100%|██████████| 3/3 [03:06<00:00, 62.17s/it]

Test loss: 0.44520 | Test accuracy: 84.53%
Train time on cpu: 186.517 seconds


In [17]:
type(train_data)

torchvision.datasets.mnist.FashionMNIST

In [33]:
image,label = test_data[1]

In [34]:
with torch.inference_mode():
  model.eval()
  pred = model(image.unsqueeze(dim=0))

In [35]:
pred

tensor([[  3.4346,  -2.2514,   7.5364,   1.1181,   4.4684,  -6.2545,   6.2373,
         -10.9770,  -0.8240,  -9.7686]])

In [36]:
torch.argmax(torch.softmax(pred,dim=1))

tensor(2)